In [42]:
import numpy as np
import pandas as pd

In [43]:
dataset = pd.read_csv("../data/training_data.csv")

In [44]:
dataset.columns = dataset.columns.str.strip()  # Removes leading and trailing spaces
print(dataset.isnull().sum()) # Check for missing values
print("---")
print(dataset.duplicated().sum()) # Check for duplicate values


step              0
type              0
amount            0
nameOrig          1
oldbalanceOrg     1
newbalanceOrig    1
nameDest          1
oldbalanceDest    1
newbalanceDest    1
isFraud           1
isFlaggedFraud    1
dtype: int64
---
0


In [45]:
dataset.dropna(inplace=True)
dataset.drop_duplicates(inplace=True)

In [46]:
dataset = dataset[[ 'amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest','isFraud']] 
dataset

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,386385.08,4669568.85,5055953.92,506035.06,119649.98,0.0
1,212458.78,234635.00,447093.78,806037.88,593579.10,0.0
2,19967.60,3634.00,0.00,0.00,0.00,0.0
3,527616.51,180216.00,0.00,92157.10,619773.61,0.0
4,206067.85,0.00,0.00,2131494.48,2337562.32,0.0
...,...,...,...,...,...,...
143920,37377.47,26397.78,0.00,753020.04,790397.50,0.0
143921,184782.46,1512205.59,1696988.05,293982.59,109200.13,0.0
143922,31391.33,31152.00,0.00,0.00,0.00,0.0
143923,255521.98,1237508.67,1493030.65,768396.97,512874.99,0.0


In [47]:
'''
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Assuming your dataset is already loaded, e.g.:
# dataset = pd.read_csv("your_file.csv")

# Select independent features and dependent target
X = dataset[['amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']] # Features
Y = dataset['isFraud'] # Target

# Create a new DataFrame that includes both X and Y for plotting
data_with_target = X.copy()  # Copy X to avoid modifying it directly
data_with_target['isFraud'] = Y  # Add the target 'isFraud' to the DataFrame

# Plot pairwise scatter plots for each feature in X against Y
for col in X.columns:
    plt.figure(figsize=(6, 4))
    sns.scatterplot(x=col, y='isFraud', data=data_with_target)
    plt.title(f'Scatter plot of {col} vs isFraud')
    plt.show()
'''

'\nimport seaborn as sns\nimport matplotlib.pyplot as plt\nimport pandas as pd\n\n# Assuming your dataset is already loaded, e.g.:\n# dataset = pd.read_csv("your_file.csv")\n\n# Select independent features and dependent target\nX = dataset[[\'amount\',\'oldbalanceOrg\',\'newbalanceOrig\',\'oldbalanceDest\',\'newbalanceDest\']] # Features\nY = dataset[\'isFraud\'] # Target\n\n# Create a new DataFrame that includes both X and Y for plotting\ndata_with_target = X.copy()  # Copy X to avoid modifying it directly\ndata_with_target[\'isFraud\'] = Y  # Add the target \'isFraud\' to the DataFrame\n\n# Plot pairwise scatter plots for each feature in X against Y\nfor col in X.columns:\n    plt.figure(figsize=(6, 4))\n    sns.scatterplot(x=col, y=\'isFraud\', data=data_with_target)\n    plt.title(f\'Scatter plot of {col} vs isFraud\')\n    plt.show()\n'

In [48]:
X = dataset[['amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']] # Features (independent)
Y = dataset['isFraud'] # Target (dependent)
X = pd.DataFrame(X)
Y = pd.DataFrame(Y)

In [49]:
print(X.columns)
print(X[X.columns[0]].value_counts().to_dict())
print(np.unique(X[X.columns[1]]))

Index(['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest',
       'newbalanceDest'],
      dtype='object')
{10000000.0: 69, 10669.76: 3, 3295.52: 3, 6687.22: 3, 9443.02: 3, 1331.11: 3, 71.61: 3, 16717.99: 3, 10000.0: 3, 2759.37: 3, 6313.43: 3, 5797.83: 3, 3851.25: 2, 1145.7: 2, 5129.87: 2, 813.26: 2, 1142.74: 2, 3662.51: 2, 3674.65: 2, 1725.45: 2, 1919.83: 2, 3065.63: 2, 3805.44: 2, 1597.08: 2, 2467.07: 2, 1124.27: 2, 188257.69: 2, 14351.3: 2, 15887.14: 2, 476.82: 2, 3213.56: 2, 11038.05: 2, 4526.6: 2, 9597.95: 2, 1233.27: 2, 6734.57: 2, 6412.24: 2, 9071.56: 2, 14105.66: 2, 3921.56: 2, 3593.99: 2, 85969.76: 2, 10226.44: 2, 1043.75: 2, 386555.11: 2, 18210.37: 2, 4153.37: 2, 11269.27: 2, 4510.8: 2, 5009.01: 2, 6478.16: 2, 348.77: 2, 7258.17: 2, 105671.23: 2, 1905.05: 2, 14103.71: 2, 175104.43: 2, 6913.1: 2, 7083.19: 2, 23312.57: 2, 42437.6: 2, 1669.65: 2, 2275.12: 2, 93584.56: 2, 6651.65: 2, 6072.61: 2, 4552.73: 2, 502.38: 2, 13931.36: 2, 7289.57: 2, 19753.44: 2, 3880.83: 2, 2

In [50]:
target_dict = Y.value_counts().to_dict()
n_labels = len(target_dict)
print(np.unique(Y))
print(target_dict)
print(n_labels)
print(type(Y)==pd.DataFrame)

[0. 1.]
{(0.0,): 143733, (1.0,): 192}
2
True


In [51]:
print(X.shape)
print(Y.shape)

(143925, 5)
(143925, 1)


In [52]:
X.iloc[1:3]


,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
1,212458.78,234635.0,447093.78,806037.88,593579.1
2,19967.60,3634.0,0.00,0.00,0.0


In [53]:
def split_indices(x_col,att):
    left_id = np.argwhere(x_col<= att).flatten()
    right_id = np.argwhere(x_col > att).flatten()

    return left_id,right_id

left_s,right_s = split_indices(X[X.columns[1]],3.00000000e-02 )

print(X[X.columns[1]])
print(left_s)
print("---")
print(right_s)

0         4669568.85
1          234635.00
2            3634.00
3          180216.00
4               0.00
             ...    
143920      26397.78
143921    1512205.59
143922      31152.00
143923    1237508.67
143924      31562.00
Name: oldbalanceOrg, Length: 143925, dtype: float64
[     4     10     12 ... 143908 143911 143919]
---
[     0      1      2 ... 143922 143923 143924]


In [54]:
print(X['amount']>200000)
left = X[(X['amount']>200000)==True]
right = X[(X['amount']>200000)==False]
print(left)
print(right)

0          True
1          True
2         False
3          True
4          True
          ...  
143920    False
143921    False
143922    False
143923     True
143924    False
Name: amount, Length: 143925, dtype: bool
           amount  oldbalanceOrg  newbalanceOrig  oldbalanceDest  \
0       386385.08     4669568.85      5055953.92       506035.06   
1       212458.78      234635.00       447093.78       806037.88   
3       527616.51      180216.00            0.00        92157.10   
4       206067.85           0.00            0.00      2131494.48   
9       284341.53     8133378.48      8417720.01      5702766.00   
...           ...            ...             ...             ...   
143903  381436.31           0.00            0.00       423565.96   
143910  253381.28       14703.00       268084.28            0.00   
143913  227101.50     1267870.75      1494972.24       808471.77   
143916  597971.62        2882.63            0.00      1297850.88   
143923  255521.98     1237508.67  

In [68]:
# Find column indices and all potential splits at said indies
def potential_splits(data):
    splits = {}
    n_col = data.shape[1]
    for i in range(n_col - 1):  # Exclude last column (target)
        splits[data.columns[i]] = np.unique(data[data.columns[i]])  # Store column names
    
    return splits

potential_splits(dataset)

{'amount': array([0.00000000e+00, 3.00000000e-02, 8.70000000e-01, ...,
        3.77966840e+07, 3.87717970e+07, 4.92039836e+07]),
 'oldbalanceOrg': array([0.00000000e+00, 1.00000000e+00, 2.00000000e+00, ...,
        3.38298204e+07, 3.43357151e+07, 3.44006513e+07]),
 'newbalanceOrig': array([0.00000000e+00, 1.76000000e+00, 5.40000000e+00, ...,
        3.38313591e+07, 3.44020062e+07, 3.44519209e+07]),
 'oldbalanceDest': array([0.00000000e+00, 1.90000000e+01, 2.30000000e+01, ...,
        1.45469306e+08, 1.71939084e+08, 1.92278926e+08]),
 'newbalanceDest': array([0.00000000e+00, 2.40500000e+01, 6.13900000e+01, ...,
        1.45494143e+08, 1.71996480e+08, 1.92307118e+08])}

In [75]:
# Function to split data based on column name and value
def split_data(data, split_col, split_val):
    left_data = data[data[split_col] <= split_val]
    right_data = data[data[split_col] > split_val]
    
    return left_data, right_data

#split_data(dataset,'oldbalanceDest',50000)

In [57]:
# Subsetting dataset
s_dict = dataset['isFraud'].value_counts().to_dict()
for k,v in s_dict.items():
    print(k,"-->",v)

subset= dataset[dataset['isFraud']==1.0]
subset.head()

0.0 --> 143733
1.0 --> 192


,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
550,1735466.52,1735466.52,0.00,0.00,0.00,1.0
636,316877.85,316877.85,0.00,0.00,0.00,1.0
864,78810.30,78810.30,0.00,802788.52,881598.82,1.0
870,513082.74,513082.74,0.00,32452.79,545535.53,1.0
1472,2542664.27,2542664.27,2542664.27,0.00,0.00,1.0


In [58]:
# Get probabilities faster
target = dataset[dataset.columns[-1]]
counts = []
p_dict = target.value_counts().to_dict()
for k,v in p_dict.items():
    counts.append(v)
counts = np.array(counts) # Allows element-wise operations
print(counts.sum())

143925


In [89]:
# NEW calculate entropy 

def entropy(data): # Takes dataset instead of column
    target_col = data[data.columns[-1]]  # Makes last column of dataset the target column
    counts = []
    target_dict = target_col.value_counts().to_dict()
    for k,v in target_dict.items():
        counts.append(v)
    counts = np.array(counts)

    p = counts / counts.sum() # Does calculation at once, so faster
    ent = -np.sum(p * np.log2(p))

    return ent

entropy(dataset)

np.float64(0.014663273789461197)

In [60]:
# NEW calculate entropy of children

def total_entropy(left_child, right_child):
    total = len(left_child) + len(right_child)

    p_left = len(left_child)/total
    p_right = len(right_child)/total

    ent_tot = (p_left * entropy(left_child)) + (p_right * entropy(right_child)) # Can use asame ent func. this since entropy now takes subset of dataset

    return ent_tot

total_entropy(dataset,dataset)

np.float64(0.014663273789461197)

In [95]:
# Get best column and attribute in that column to split at
def get_best_split(data, pot_splits):
    best_ent = np.inf  # Initialize entropy
    best_col = None    # Initialize best column
    best_val = None    # Initialize best value

    for k in pot_splits:
        for v in pot_splits[k]:
            left_child, right_child = split_data(data, k, v)
            current_ent = total_entropy(left_child, right_child)

            if current_ent < best_ent:
                best_ent = current_ent
                best_col = k
                best_val = v

    return best_col, best_val

potential_splits = potential_splits(dataset)
get_best_split(dataset,potential_splits)

TypeError: 'dict' object is not callable

In [99]:
dataset[dataset.columns[-1]].unique()[1]
dataset.head()


,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,386385.08,4669568.85,5055953.92,506035.06,119649.98,0.0
1,212458.78,234635.00,447093.78,806037.88,593579.10,0.0
2,19967.60,3634.00,0.00,0.00,0.00,0.0
3,527616.51,180216.00,0.00,92157.10,619773.61,0.0
4,206067.85,0.00,0.00,2131494.48,2337562.32,0.0


In [104]:
def decision_tree(df, min_samples):
    if type(df) != type(df.values): # To convert to np array
        data = df.values
    else:
        data = df
    
    depth = 0
    target = data[data.columns[-1]]
    if (len(np.unique(target)) == 1) or (len(data)<=min_samples):
        return target.unique()[0]
    
    else: 
        depth +=1
        # Determining the child nodes
        pot_splits = potential_splits(data)
        split_col, split_val = get_best_split(data, pot_splits)
        left_child, right_child = split_data(data, split_col, split_val)

        # Making the tree
        cond = f"{split_col} <= {split_val}"
        sub_tree = {cond: []} # Tree is in form of dictionary

        node_yes = decision_tree(left_child, min_samples)
        node_no = decision_tree(right_child, min_samples)

        sub_tree[cond].append[node_yes]
        sub_tree[cond].append[node_no]

        return sub_tree

decision_tree(dataset.head(),100)
    
    

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [ ]:
# Entropy calculation for an attribute in a column
'''
def entropy_attribute(col,val,target):
    subset = dataset[dataset[col]==val] # Make subset based on attribute 
    
    ent = 0 # Initialize entropy value

    target_dict = subset[target].value_counts().to_dict()
    for k,v in target_dict.items():
        p = v/len(subset) # Occurences of 1s/0s divided by len of column
        ent -= p * np.log2(p) # Entropy of attribute

    return ent

entropy_attribute('isFraud',1.0,'isFraud')
'''

np.float64(0.0)

In [ ]:
# Entropy of parent
'''
def entropy_dataset(target):
    target_dict = dataset[target].value_counts().to_dict()
    ent = 0
    for k,v in target_dict.items():
        p = v/len(dataset)
        ent -= p * np.log2(p)
    
    return ent

entropy_dataset('isFraud')
'''

np.float64(0.014663273789461197)

In [76]:
# Information Gain Calculation
'''
def information_gain(col,target):
    len_data = len(dataset) # Length of dataset
    att_dict = dataset[col].value_counts().to_dict() # Make dictionary of unique attribute and no. of occurences

    ent_data = entropy_dataset(target) # Calculate entropy of dataset
    ent_att = 0 # Initalize entropy of attributes of a column

    for k,v in att_dict.items():
        p = v / len_data # No. of occurences of attributes / length of dataset
        ent_att -= p * entropy_attribute(col,k,target) # Sum of entropy of all attributes in a column
    
    info_gain = ent_data + ent_att # Calculate information gain

    return info_gain

information_gain('type','isFraud')
'''

"\ndef information_gain(col,target):\n    len_data = len(dataset) # Length of dataset\n    att_dict = dataset[col].value_counts().to_dict() # Make dictionary of unique attribute and no. of occurences\n\n    ent_data = entropy_dataset(target) # Calculate entropy of dataset\n    ent_att = 0 # Initalize entropy of attributes of a column\n\n    for k,v in att_dict.items():\n        p = v / len_data # No. of occurences of attributes / length of dataset\n        ent_att -= p * entropy_attribute(col,k,target) # Sum of entropy of all attributes in a column\n    \n    info_gain = ent_data + ent_att # Calculate information gain\n\n    return info_gain\n\ninformation_gain('type','isFraud')\n"